In [1]:
import pandas as pd
import numpy as np

-----

In [78]:
## データ数が少ないのでランダムでぜんぜん成績が変わる
df = pd.read_csv("./train.csv")
df.shape, df.shape

((891, 12), (891, 12))

In [79]:
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
PassengerId,891,NaN,NaN,NaN,446,257.354,1,223.5,446,668.5,891
Survived,891,NaN,NaN,NaN,0.383838,0.486592,0,0,0,1,1
Pclass,891,NaN,NaN,NaN,2.30864,0.836071,1,2,3,3,3
Name,891,891,"Gustafsson, Mr. Johan Birger",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sex,891,2,male,577,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,714,NaN,NaN,NaN,29.6991,14.5265,0.42,20.125,28,38,80
SibSp,891,NaN,NaN,NaN,0.523008,1.10274,0,0,0,1,8
Parch,891,NaN,NaN,NaN,0.381594,0.806057,0,0,0,0,6
Ticket,891,681,CA. 2343,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fare,891,NaN,NaN,NaN,32.2042,49.6934,0,7.9104,14.4542,31,512.329


In [7]:
df
# Port of Embarkation 乗船港
# C＝シェルブール
# Q＝クイーンズタウン
# S＝サウサンプトン

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


----

### lightbgm

In [30]:
import lightgbm as lgb

def exec_lightgbm(
    trainX_,
    trainY_,
    validX_,
    validY_,
    testX_,
    testY_,
    ):

    # trainの X と Y を専用の箱に入れる
    lgb_train = lgb.Dataset( trainX_ , trainY_ )
    
    # testの X と Y を専用の箱に入れる
    lgb_valid = lgb.Dataset( validX_ , validY_)
    
    
    param = {'num_leaves': 64, 'objective': 'binary'}
    param['metric'] = 'auc'
    num_round = 1000
    
    bst = lgb.train(param,
                    lgb_train,
                    num_round,
                    valid_sets=[lgb_valid],
                    early_stopping_rounds=10,
                    verbose_eval=False)
    
    
    from sklearn import metrics
    predY_ = bst.predict(testX_)
    score = metrics.roc_auc_score(testY_, predY_)
    print(f"Test AUC score: {score}")

------------

### TargetEncoder

In [64]:
from sklearn.model_selection import train_test_split
trainX, validX, trainY, validY = train_test_split(df_X, df_Y, test_size=0.2)
trainX = trainX.drop(["PassengerId", "Name","Ticket","Cabin"], axis=1)
validX = validX.drop(["PassengerId", "Name","Ticket","Cabin"], axis=1)

In [65]:
import category_encoders

cat_col = ["Sex", "Embarked"]

ce = category_encoders.TargetEncoder(cols=cat_col)
ce.fit(trainX[cat_col], trainY)

trainX = trainX.join( ce.transform( trainX[cat_col] ).add_suffix("_ce") )
validX = validX.join( ce.transform( validX[cat_col] ).add_suffix("_ce") )

trainX = trainX.drop( cat_col, axis=1 )
validX = validX.drop( cat_col, axis=1 )

In [66]:
testX = validX[:90]
testY = validY[:90]
validX = validX[90:]
validY = validY[90:]
validX.shape, validY.shape, testX.shape, testY.shape

((89, 7), (89, 1), (90, 7), (90, 1))

In [67]:
exec_lightgbm(
    trainX,
    trainY,
    validX,
    validY,
    testX,
    testY)

Test AUC score: 0.8603238866396762


--------------

### CatBoostEncoder

In [68]:
from sklearn.model_selection import train_test_split
trainX, validX, trainY, validY = train_test_split(df_X, df_Y, test_size=0.2)
trainX = trainX.drop(["PassengerId", "Name","Ticket","Cabin"], axis=1)
validX = validX.drop(["PassengerId", "Name","Ticket","Cabin"], axis=1)

In [69]:
import category_encoders

cat_col = ["Sex", "Embarked"]

ce = category_encoders.CatBoostEncoder(cols=cat_col)
ce.fit(trainX[cat_col], trainY)

trainX = trainX.join( ce.transform( trainX[cat_col] ).add_suffix("_ce") )
validX = validX.join( ce.transform( validX[cat_col] ).add_suffix("_ce") )

trainX = trainX.drop( cat_col, axis=1 )
validX = validX.drop( cat_col, axis=1 )

In [70]:
testX = validX[:90]
testY = validY[:90]
validX = validX[90:]
validY = validY[90:]
# validX.shape, validY.shape, testX.shape, testY.shape

In [71]:
exec_lightgbm(
    trainX,
    trainY,
    validX,
    validY,
    testX,
    testY)

Test AUC score: 0.8808198380566801


---------------

### CountEncoder

In [72]:
from sklearn.model_selection import train_test_split
trainX, validX, trainY, validY = train_test_split(df_X, df_Y, test_size=0.2)
trainX = trainX.drop(["PassengerId", "Name","Ticket","Cabin"], axis=1)
validX = validX.drop(["PassengerId", "Name","Ticket","Cabin"], axis=1)

In [73]:
import category_encoders

cat_col = ["Sex", "Embarked"]

ce = category_encoders.CountEncoder(cols=cat_col)
ce.fit(trainX[cat_col], trainY)

trainX = trainX.join( ce.transform( trainX[cat_col] ).add_suffix("_ce") )
validX = validX.join( ce.transform( validX[cat_col] ).add_suffix("_ce") )

trainX = trainX.drop( cat_col, axis=1 )
validX = validX.drop( cat_col, axis=1 )

In [74]:
testX = validX[:90]
testY = validY[:90]
validX = validX[90:]
validY = validY[90:]
# validX.shape, validY.shape, testX.shape, testY.shape

In [75]:
exec_lightgbm(
    trainX,
    trainY,
    validX,
    validY,
    testX,
    testY)

Test AUC score: 0.7678571428571428


---------------

In [105]:
from sklearn.model_selection import train_test_split
trainX, validX, trainY, validY = train_test_split(df_X, df_Y, test_size=0.2)
trainX = trainX.drop(["PassengerId", "Name","Ticket","Cabin"], axis=1)
validX = validX.drop(["PassengerId", "Name","Ticket","Cabin"], axis=1)

In [103]:
from sklearn.preprocessing import OneHotEncoder

cat_col = ["Sex", "Embarked"]

for col in cat_col:
    trainX[col].fillna("null", inplace=True)
    validX[col].fillna("null", inplace=True)

ce = OneHotEncoder(handle_unknown='ignore',sparse=False)
ce.fit(trainX[cat_col])

trainX_oh = trainX.join( pd.DataFrame(ce.transform( trainX[cat_col] )).add_suffix("_ce") )
validX_oh = validX.join( pd.DataFrame(ce.transform( validX[cat_col] )).add_suffix("_ce") )


In [104]:
trainX_oh = trainX.join( pd.DataFrame(ce.transform( trainX[cat_col] )).add_suffix("_ce") )
validX_oh = validX.join( pd.DataFrame(ce.transform( validX[cat_col] )).add_suffix("_ce") )

trainX = trainX.drop( cat_col, axis=1 )
validX = validX.drop( cat_col, axis=1 )

,Pclass,Age,SibSp,Parch,Fare,0_ce,1_ce,2_ce,3_ce,4_ce,5_ce
836,3,21.0,0,0,8.6625,NaN,NaN,NaN,NaN,NaN,NaN
397,2,46.0,0,0,26.0000,1.0,0.0,0.0,0.0,1.0,0.0
660,1,50.0,2,0,133.6500,1.0,0.0,1.0,0.0,0.0,0.0
175,3,18.0,1,1,7.8542,0.0,1.0,0.0,0.0,1.0,0.0
278,3,7.0,4,1,29.1250,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
606,3,30.0,0,0,7.8958,1.0,0.0,0.0,1.0,0.0,0.0
269,1,35.0,0,0,135.6333,0.0,1.0,0.0,0.0,1.0,0.0
304,3,NaN,0,0,8.0500,0.0,1.0,0.0,0.0,1.0,0.0
459,3,NaN,0,0,7.7500,1.0,0.0,0.0,1.0,0.0,0.0


In [98]:
trainX["Sex"].value_counts(dropna=False)

male      453
female    259
Name: Sex, dtype: int64

In [99]:
trainX["Embarked"].value_counts(dropna=False)

S       512
C       135
Q        63
null      2
Name: Embarked, dtype: int64

In [94]:
trainX["Embarked"].fillna("null", inplace=True)